# Simulation Migration Visualisation
This notebook creates a visualisation of migration events that have been accepted during the simulation. This does not work offline (aka cannot be automated) due to a weird feature about Plotly and Geopandas, hence why this is a notebook.
To create visualisation for a specific script do, change the "sim_path" variable below to the absolute folder path of the SuperSimPy output.

In [14]:
sim_path = "../output/"

## Collate data
This section just collates data and puts it into a format that can be used to graph migrations.

In [15]:
import pandas as pd
import plotly.graph_objects as go
import geopandas

effective_distances = pd.read_csv('../pre_processing/data_sources/effective.distance.matrix.country.csv', header=0)
#World
world = geopandas.read_file('data_sources/ne_110m_admin_0_countries.zip')
alpha2_codes = effective_distances.columns
effective_distances.index = alpha2_codes

country_codes = pd.read_csv('data_sources/countries_codes_and_coordinates.csv')
country_codes['Alpha-2 code'] = country_codes['Alpha-2 code'].str.replace('"', '')
coords = {}
for alpha2 in alpha2_codes:
    row = country_codes.loc[country_codes['Alpha-2 code'] == " " + alpha2]
    lon = float(row['Longitude (average)'].values[0].replace(' ', '').replace('"', ''))
    lat = float(row['Latitude (average)'].values[0].replace(' ', '').replace('"', ''))
    coords[alpha2] = [lon, lat]
dictionary = pd.read_csv('../parameters/manypop_country_ids.csv')
migrations = pd.read_csv(f"{sim_path}migrations.tsv", sep='\t',
                         dtype=dict(location=int, fullname=str, alpha2=str, population=int))
dictionary = dictionary[['alpha2', 'location']]
mergea = dictionary.add_suffix('_old').merge(migrations, left_on='location_old', right_on='Old_population')
mergeb = dictionary.add_suffix('_new').merge(mergea, left_on='location_new', right_on='New_population')
alpha2_migrations = mergeb[['alpha2_new','alpha2_old', 'Time', 'Node']]
migration_counts = (alpha2_migrations.groupby(['alpha2_new','alpha2_old']).size())

ValueError: Length mismatch: Expected axis has 228 elements, new values have 229 elements

## Plot graph

In [ ]:
fig = go.Figure()
maximum = migration_counts.max()
for key, count in migration_counts.items():
    alpha_old, alpha_new = key
    coord_old, coord_new = coords[alpha_old], coords[alpha_new]
    fig.add_trace(go.Scattergeo(locationmode='ISO-3', lon=[coord_old[0], coord_new[0]], lat=[coord_old[1], coord_new[1]],
                  mode='lines', line=dict(width=1.5*float(count / maximum), color='red'),
                  opacity=1))
fig.update_layout(
    title_text='Simulation accepted migration events',
    title_x=0.5,
    showlegend=False,
    geo=dict(scope="world",
             showland=True,landcolor='rgb(243,243,243)',
             countrycolor='rgb(204,204,204)',projection=dict(type="mercator")),
    autosize=False,
    width= 1000,
    height= 600,
)
fig.show()